**This notebook contains a script that creates saves and tracks POE documents**

**Add local Library to Path**

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(module_path + '/local_library')

**Import Library**

In [3]:
import pandas as pd
import re
from local_library import create_folder
from local_library import import_worksheet
from local_library import export_worksheet
from local_library import get_documents_metadata

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1017190226189-f1d5s7cpjrj54u2rqk1ufh9pevguqoap.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


**Import Data**

In [9]:
#import cohort data
cohort_df = import_worksheet("Cohort 2020 Data", "Cohort 2020 Data")

**Update Column Names**

In [10]:
column_names = {"Firstname":"learner_name", "Surname": "learner_surname",
                "Student Email":"learner_email", "Middle Name/s":"middle_name"
               }

cohort_df = cohort_df.rename(columns=column_names)

**Create Fullname**

In [11]:
cohort_df['middle_name'] = cohort_df['middle_name'].fillna("")
cohort_df['learner_name'] = cohort_df['learner_name'].str.strip().str.lower() + " " + cohort_df['middle_name'].str.strip().str.lower()
cohort_df['learner_name'] = cohort_df['learner_name'].str.strip().str.title()
cohort_df['learner_fullname'] = cohort_df['learner_name'].str.strip().str.lower() + " " + cohort_df['learner_surname'].str.strip().str.lower()
cohort_df['learner_fullname'] = [re.sub(' +', ' ', fullname) for fullname in cohort_df['learner_fullname']]
cohort_df['learner_fullname'] = cohort_df['learner_fullname'].str.title()


**Subset Useful Columns**

In [12]:
column_names = ['Username', 'Campus', 'learner_email', 'learner_fullname']

cohort_df = cohort_df[cohort_df['Active / Not active/Early absorption'] == 'Active']
cohort_df = cohort_df[column_names]

**Create Folder ID Column**

In [13]:
cohort_df['folder_id'] = ''
len(cohort_df)

257

**Create Folders**

In [14]:
#for index, row in cohort_df.iterrows():
#    parent_id = create_folder("1W04tIGaWpqSgKsaP6xSjrUtP1OsVcaPX",row['learner_fullname'])
#    cohort_df.loc[cohort_df['learner_fullname'] == row['learner_fullname'], 'folder_id'] = parent_id

**Add template IDS**

In [15]:
cohort_df['poe_instructions'] = 'feb67241-52f8-456c-917e-bdd0526619ee'
cohort_df['learner_contract'] = '900be1ba-0b96-4386-a904-fb67069f168e'
cohort_df['appeals_procedure'] = '589d226b-1c7e-441f-b99e-1d30a2d37761'
cohort_df.loc[cohort_df['Campus'] == 'Johannesburg', 'assessment_contract'] = 'b48127ae-d278-467a-89b6-f6b3f010edb4'
cohort_df.loc[cohort_df['Campus'] == 'Cape Town', 'assessment_contract'] = '9a6b58fc-a6f6-4fba-8508-4ef859d98088'
cohort_df.loc[cohort_df['Campus'] == 'Johannesburg', 'preassessment_meeting'] = '2cec54b7-0ce3-4bd2-a9c5-29badf20d4f4'
cohort_df.loc[cohort_df['Campus'] == 'Cape Town', 'preassessment_meeting'] = '83797d10-f3d4-409d-aa39-93565ccd60fd'
cohort_df.loc[cohort_df['Campus'] == 'Durban', 'assessment_contract'] = '9a6b58fc-a6f6-4fba-8508-4ef859d98088'
cohort_df.loc[cohort_df['Campus'] == 'Durban', 'preassessment_meeting'] = '83797d10-f3d4-409d-aa39-93565ccd60fd'

In [16]:
#Declaration of Authenticity
#cohort_df['authenticity_declaration_status'] = ''
cohort_df.loc[cohort_df['Campus'] == 'Johannesburg', 'authenticity_declaration'] = 'd2308911-b247-4957-b865-e76f504ef91f'
cohort_df.loc[cohort_df['Campus'] != 'Johannesburg', 'authenticity_declaration'] = '78c858be-3acf-4c37-987d-672536c3196e'

In [17]:
#Program Evaluation
#cohort_df['program_evaluation_status'] = ''
cohort_df.loc[cohort_df['Campus'] == 'Johannesburg', 'program_evaluation'] = 'ecae0443-8f3c-44dd-b2ee-7194a4fde67b'
cohort_df.loc[cohort_df['Campus'] != 'Johannesburg', 'program_evaluation'] = 'f97eb0a8-97a9-40c7-84df-29aaee7e9893'

In [18]:
#Logbook
#cohort_df['logbook_status'] = ''
cohort_df.loc[cohort_df['Campus'] == 'Johannesburg', 'logbook'] = 'f8c63d3d-fd0f-413f-a028-35dd9990b30d'
cohort_df.loc[cohort_df['Campus'] != 'Johannesburg', 'logbook'] = '911ce292-ec38-4ef9-a8a0-b48b1adbef5f'

**Save Database to Spreadsheet**

In [19]:
#export_worksheet("POE Compilation","POE Documents", cohort_df)

**Import POE Template Data**

In [5]:
cohort_df = import_worksheet("POE Compilation","Follow")

**Run Main Function**

In [6]:
#create document names            
document_names = ['preassessment_meeting', 'authenticity_declaration', 'poe_instructions', 'learner_contract','appeals_procedure', 'assessment_contract','program_evaluation', 'logbook']

for document_name in document_names:
    get_documents_metadata(cohort_df, document_name, "learner_email")
    print(document_name)

preassessment_meeting
authenticity_declaration
poe_instructions
learner_contract
appeals_procedure
assessment_contract
program_evaluation
logbook


**Save to POE tracking sheet**

In [7]:
#export_worksheet("2020 SETA Submission Tracking Sheet","POE Documents Data Dump", cohort_df[['Username', 'Campus',
#                'poe_instructions_status', 'learner_contract_status', 'appeals_procedure_status', 'assessment_contract_status',
#                'preassessment_meeting_status', 'authenticity_declaration_status', 'program_evaluation_status', 'logbook_status']])


**Save to POE Follow sheet**

In [8]:
#export_worksheet("POE Compilation","POE Follow", cohort_df)
